### Задание на ПР-7:
В приложенном датасете приведены данные о реальных больных сердечно-сосудистыми заболеваниями. Последняя компонента в каждом экземпляре - бинарная метка (0 - нет болезни, 1 - есть болезнь). Ваша задача - построить классификатор, который на данных нового пациента предскажет наличие у него болезни. Затем нужно вычислить метрики качества вашего классификатора. 
1. Сделайте препроцессинг.
2. Разделите данные на обучающие и тестовые (70/30), причем проследите чтобы пропорции 0 и 1 в них были примерно одинаковыми.
3. Напишите функцию, которая вычислит коэффициенты логистической регрессии на обучающей выборке.
4. Напишите функцию, которая предскажет метку на тестовом экземпляре данных, пользуясь вычисленными коэффициентами логистической регрессии (ЛР).
5. Выберите из датасета такие экземпляры, на которых логистическая регрессия дает нестабильные предсказания. Таких экземпляров должно быть не менее 20% от всего датасета. Отметьте их третьим типом метки (например, -1, или 2 - как вам нравится больше).
6. Постройте классификацию (выполните пп. 3 и 4) методом ближайших соседей (БС). 
7. Измерьте качество классификации для ЛР и БС по метрикам Precision, Recall, F1 (для бинарной классификации) и Accuracy (для тернарной классификации). 
8. Интерпретируйте результаты - сделайте выводы о качестве классификации, проведенной на основе разных методов и с разным числом классов.

In [1]:
# Импорт библиотек и базовые функции
import math  # математические функции
import random  # функции для генерации случайных чисел
import csv  # функции для работы с CSV файлами

# Функция для скалярного произведения двух векторов
def dot_product(vector1, vector2):
    """Вычисляет скалярное произведение двух векторов одинаковой длины"""
    result = 0.0  # Инициализируем переменную для результата
    # Проходим по всем элементам векторов
    for element_index in range(len(vector1)):
        # Умножаем соответствующие элементы и добавляем к результату
        result = result + vector1[element_index] * vector2[element_index]
    return result  # Возвращаем скалярное произведение

# Функция для сложения двух векторов
def vector_add(vector1, vector2):
    """Поэлементное сложение двух векторов одинаковой длины"""
    result_vector = []  # Создаем пустой список для результата
    # Проходим по всем элементам векторов
    for element_index in range(len(vector1)):
        # Складываем соответствующие элементы и добавляем в результат
        result_vector.append(vector1[element_index] + vector2[element_index])
    return result_vector  # Возвращаем результирующий вектор

# Функция для умножения вектора на скаляр
def scalar_multiply(scalar, vector):
    """Умножает каждый элемент вектора на скаляр"""
    result_vector = []  # Создаем пустой список для результата
    # Проходим по всем элементам вектора
    for element_index in range(len(vector)):
        # Умножаем элемент на скаляр и добавляем в результат
        result_vector.append(scalar * vector[element_index])
    return result_vector  # Возвращаем результирующий вектор

# Функция для вычисления градиента
def gradient(function_to_diff, point, step_size=1e-05):
    """Вычисляет градиент функции в заданной точке с использованием конечных разностей"""
    gradient_vector = []  # Создаем пустой список для градиента
    # Проходим по всем координатам точки
    for coordinate_index in range(len(point)):
        perturbed_point = []  # Создаем возмущенную точку
        # Создаем копию точки с возмущением по текущей координате
        for coordinate_index_inner in range(len(point)):
            if coordinate_index_inner == coordinate_index:
                # Добавляем шаг к текущей координате
                perturbed_point.append(point[coordinate_index_inner] + step_size)
            else:
                # Оставляем координату без изменений
                perturbed_point.append(point[coordinate_index_inner])
        # Вычисляем частную производную как разностное отношение
        partial_derivative = (function_to_diff(perturbed_point) - function_to_diff(point)) / step_size
        gradient_vector.append(partial_derivative)  # Добавляем в градиент
    return gradient_vector  # Возвращаем вектор градиента

# Функция градиентного спуска
def gradient_descent(function_to_minimize, initial_point, learning_rate=0.05, step_size=1e-05, max_iterations=5000):
    """Реализация градиентного спуска для минимизации функции"""
    current_point = []  # Создаем список для текущей точки
    # Копируем начальную точку
    for value in initial_point:
        current_point.append(value)
    
    # Выполняем итерации градиентного спуска
    for iteration in range(max_iterations):
        # Вычисляем градиент в текущей точке
        current_gradient = gradient(function_to_minimize, current_point, step_size)
        # Вычисляем норму градиента
        gradient_norm = 0.0
        for gradient_component in current_gradient:
            gradient_norm = gradient_norm + gradient_component * gradient_component
        gradient_norm = math.sqrt(gradient_nradient_norm)
        # Масштабируем градиент на скорость обучения
        scaled_gradient = scalar_multiply(learning_rate * gradient_norm, current_gradient)
        # Делаем шаг в направлении, противоположном градиенту
        current_point = vector_add(current_point, scalar_multiply(-1, scaled_gradient))
    return current_point  # Возвращаем найденную точку минимума

#### ЗАДАНИЕ 1: ПРЕПРОЦЕССИНГ ДАННЫХ

In [2]:
# Загрузка и препроцессинг данных
def load_data(filename):
    """Загружает данные из CSV файла и возвращает список словарей"""
    data_list = []  # Создаем пустой список для данных
    # Открываем файл для чтения
    with open(filename, 'r') as file_handle:
        # Создаем CSV reader для чтения данных
        csv_reader = csv.DictReader(file_handle)
        # Читаем каждую строку файла
        for row_dict in csv_reader:
            data_list.append(row_dict)  # Добавляем строку в список данных
    return data_list  # Возвращаем загруженные данные

def preprocess_data(raw_data):
    """Преобразует категориальные признаки в числовые значения"""
    processed_data_list = []  # Создаем пустой список для обработанных данных
    
    # Создаем словари для преобразования категориальных признаков
    gender_mapping = {'M': 1, 'F': 0}  # Мужской пол -> 1, женский -> 0
    chest_pain_mapping = {'ATA': 0, 'NAP': 1, 'ASY': 2, 'TA': 3}  # Типы боли в груди
    ecg_mapping = {'Normal': 0, 'ST': 1, 'LVH': 2}  # Типы ЭКГ
    angina_mapping = {'Y': 1, 'N': 0}  # Наличие стенокардии при нагрузке
    slope_mapping = {'Up': 0, 'Flat': 1, 'Down': 2}  # Наклон сегмента ST
    
    # Обрабатываем каждую строку исходных данных
    for data_row in raw_data:
        processed_row = []  # Создаем пустой список для обработанной строки
        # Преобразуем возраст в целое число
        processed_row.append(int(data_row['Age']))
        # Преобразуем пол с помощью словаря
        processed_row.append(gender_mapping[data_row['Sex']])
        # Преобразуем тип боли в груди
        processed_row.append(chest_pain_mapping[data_row['ChestPainType']])
        # Преобразуем артериальное давление в покое
        processed_row.append(int(data_row['RestingBP']))
        # Преобразуем уровень холестерина
        processed_row.append(int(data_row['Cholesterol']))
        # Преобразуем уровень сахара натощак
        processed_row.append(int(data_row['FastingBS']))
        # Преобразуем результат ЭКГ
        processed_row.append(ecg_mapping[data_row['RestingECG']])
        # Преобразуем максимальный пульс
        processed_row.append(int(data_row['MaxHR']))
        # Преобразуем наличие стенокардии при нагрузке
        processed_row.append(angina_mapping[data_row['ExerciseAngina']])
        # Преобразуем депрессию ST сегмента в вещественное число
        processed_row.append(float(data_row['Oldpeak']))
        # Преобразуем наклон ST сегмента
        processed_row.append(slope_mapping[data_row['ST_Slope']])
        # Преобразуем целевую переменную (наличие заболевания)
        processed_row.append(int(data_row['HeartDisease']))
        # Добавляем обработанную строку в список
        processed_data_list.append(processed_row)
    return processed_data_list  # Возвращаем обработанные данные

# Загружаем исходные данные из CSV файла
raw_dataset = load_data('heart.csv')
# Преобразуем категориальные признаки в числовые
processed_dataset = preprocess_data(raw_dataset)
# Выводим информацию о количестве загруженных записей
print("Загружено " + str(len(processed_dataset)) + " записей")

Загружено 918 записей


In [3]:
# Масштабирование признаков и добавление intercept
def scale_features(input_data):
    """Масштабирует признаки методом стандартизации (z-score normalization)"""
    # Проверяем, что данные не пустые
    if len(input_data) == 0:
        return input_data
    
    # Транспонируем данные для обработки по столбцам (признакам)
    transposed_data = []  # Создаем список для транспонированных данных
    # Проходим по всем столбцам (признакам)
    for column_index in range(len(input_data[0])):
        current_column = []  # Создаем список для текущего столбца
        # Собираем все значения текущего столбца
        for row_index in range(len(input_data)):
            current_column.append(input_data[row_index][column_index])
        transposed_data.append(current_column)  # Добавляем столбец в транспонированные данные
    
    scaled_transposed_data = []  # Создаем список для масштабированных данных
    
    # Обрабатываем все признаки кроме последнего (целевой переменной)
    for feature_index in range(len(transposed_data) - 1):
        feature_column = transposed_data[feature_index]  # Получаем текущий признак
        # Вычисляем среднее значение признака
        feature_sum = 0.0
        for feature_value in feature_column:
            feature_sum = feature_sum + feature_value
        feature_mean = feature_sum / len(feature_column)
        
        # Вычисляем стандартное отклонение признака
        sum_squared_diff = 0.0
        for feature_value in feature_column:
            sum_squared_diff = sum_squared_diff + (feature_value - feature_mean) * (feature_value - feature_mean)
        feature_std = math.sqrt(sum_squared_diff / len(feature_column))
        
        # Если стандартное отклонение равно 0, устанавливаем 1
        if feature_std == 0:
            feature_std = 1
        
        # Масштабируем значения признака
        scaled_feature_column = []
        for feature_value in feature_column:
            scaled_value = (feature_value - feature_mean) / feature_std
            scaled_feature_column.append(scaled_value)
        scaled_transposed_data.append(scaled_feature_column)
    
    # Добавляем целевую переменную без изменений
    scaled_transposed_data.append(transposed_data[-1])
    
    # Возвращаем данные к исходной структуре (транспонируем обратно)
    scaled_dataset = []
    for row_index in range(len(scaled_transposed_data[0])):
        new_row = []
        for column_index in range(len(scaled_transposed_data)):
            new_row.append(scaled_transposed_data[column_index][row_index])
        scaled_dataset.append(new_row)
    
    return scaled_dataset  # Возвращаем масштабированные данные

# Масштабируем признаки dataset
scaled_dataset = scale_features(processed_dataset)

# Добавляем столбец единиц для intercept (свободного члена)
dataset_with_intercept = []
for data_row in scaled_dataset:
    new_row = [1]  # Добавляем единицу в начало для intercept
    # Копируем все признаки кроме целевой переменной
    for feature_index in range(len(data_row) - 1):
        new_row.append(data_row[feature_index])
    # Добавляем целевую переменную в конец
    new_row.append(data_row[-1])
    dataset_with_intercept.append(new_row)

# Выводим информацию о размерности данных
print("Добавлен intercept, размерность данных: " + str(len(dataset_with_intercept[0])))

Добавлен intercept, размерность данных: 13


#### ЗАДАНИЕ 2: РАЗДЕЛЕНИЕ ДАННЫХ НА ОБУЧАЮЩУЮ И ТЕСТОВУЮ ВЫБОРКИ

In [4]:
# Разделение данных на обучающую и тестовую выборки
def train_test_split(input_data, test_fraction=0.3):
    """Разделяет данные на обучающую и тестовую выборки со стратификацией"""
    # Разделяем данные по классам для сохранения пропорций
    class_0_samples = []  # Создаем список для образцов класса 0 (нет болезни)
    class_1_samples = []  # Создаем список для образцов класса 1 (есть болезнь)
    
    # Распределяем образцы по классам
    for data_sample in input_data:
        if data_sample[-1] == 0:
            class_0_samples.append(data_sample)
        else:
            class_1_samples.append(data_sample)
    
    # Перемешиваем данные в каждом классе
    random.shuffle(class_0_samples)
    random.shuffle(class_1_samples)
    
    # Вычисляем количество образцов для теста в каждом классе
    test_samples_class_0 = int(len(class_0_samples) * test_fraction)
    test_samples_class_1 = int(len(class_1_samples) * test_fraction)
    
    # Формируем тестовую выборку
    test_dataset = []
    for sample_index in range(test_samples_class_0):
        test_dataset.append(class_0_samples[sample_index])
    for sample_index in range(test_samples_class_1):
        test_dataset.append(class_1_samples[sample_index])
    
    # Формируем обучающую выборку
    train_dataset = []
    for sample_index in range(test_samples_class_0, len(class_0_samples)):
        train_dataset.append(class_0_samples[sample_index])
    for sample_index in range(test_samples_class_1, len(class_1_samples)):
        train_dataset.append(class_1_samples[sample_index])
    
    # Перемешиваем итоговые выборки
    random.shuffle(test_dataset)
    random.shuffle(train_dataset)
    
    # Разделяем на признаки (X) и метки (y)
    train_features = []  # Признаки обучающей выборки
    train_labels = []    # Метки обучающей выборки
    for data_sample in train_dataset:
        feature_vector = []
        for feature_index in range(len(data_sample) - 1):
            feature_vector.append(data_sample[feature_index])
        train_features.append(feature_vector)
        train_labels.append(data_sample[-1])
    
    test_features = []   # Признаки тестовой выборки
    test_labels = []     # Метки тестовой выборки
    for data_sample in test_dataset:
        feature_vector = []
        for feature_index in range(len(data_sample) - 1):
            feature_vector.append(data_sample[feature_index])
        test_features.append(feature_vector)
        test_labels.append(data_sample[-1])
    
    return train_features, test_features, train_labels, test_labels

# Разделяем данные на обучающую и тестовую выборки
train_features, test_features, train_labels, test_labels = train_test_split(dataset_with_intercept, test_fraction=0.3)

# Подсчитываем количество образцов каждого класса в обучающей выборке
class_0_count_train = 0
class_1_count_train = 0
for label_value in train_labels:
    if label_value == 0:
        class_0_count_train = class_0_count_train + 1
    else:
        class_1_count_train = class_1_count_train + 1

# Выводим информацию о разделении данных
print("Обучающая выборка: " + str(len(train_features)) + " примеров")
print("Тестовая выборка: " + str(len(test_features)) + " примеров")
print("Пропорции классов в обучающей: 0=" + str(class_0_count_train) + ", 1=" + str(class_1_count_train))

Обучающая выборка: 643 примеров
Тестовая выборка: 275 примеров
Пропорции классов в обучающей: 0=287, 1=356


#### ЗАДАНИЕ 3: РЕАЛИЗАЦИЯ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ И ОБУЧЕНИЕ МОДЕЛИ

In [5]:
# Функции для логистической регрессии
def sigmoid(input_value):
    """Сигмоидная функция - преобразует любое число в интервал (0, 1)"""
    # Проверяем на переполнение для очень маленьких значений
    if input_value < -700:
        return 0.0
    # Проверяем на переполнение для очень больших значений
    if input_value > 700:
        return 1.0
    # Вычисляем значение сигмоидной функции
    return 1.0 / (1.0 + math.exp(-input_value))

def log_likelihood_single_sample(feature_vector, true_label, model_parameters):
    """Логарифм правдоподобия для одного наблюдения с защитой от крайних значений"""
    # Вычисляем линейную комбинацию признаков и параметров
    linear_combination = dot_product(feature_vector, model_parameters)
    # Преобразуем в вероятность с помощью сигмоидной функции
    predicted_probability = sigmoid(linear_combination)
    
    # Добавляем защиту от крайних значений вероятности
    epsilon = 1e-15  # Малое число для избежания логарифма от 0
    if predicted_probability < epsilon:
        predicted_probability = epsilon
    if predicted_probability > 1 - epsilon:
        predicted_probability = 1 - epsilon
    
    # Вычисляем логарифм правдоподобия в зависимости от истинной метки
    if true_label == 1:
        return math.log(predicted_probability)  # Логарифм P(y=1|x)
    else:
        return math.log(1 - predicted_probability)  # Логарифм P(y=0|x)

def log_likelihood_total(feature_matrix, label_vector, model_parameters):
    """Суммарный логарифм правдоподобия для всей выборки"""
    total_log_likelihood = 0.0  # Инициализируем суммарное правдоподобие
    # Проходим по всем наблюдениям в выборке
    for sample_index in range(len(feature_matrix)):
        # Вычисляем правдоподобие для текущего наблюдения
        sample_likelihood = log_likelihood_single_sample(
            feature_matrix[sample_index], 
            label_vector[sample_index], 
            model_parameters
        )
        total_log_likelihood = total_log_likelihood + sample_likelihood
    return total_log_likelihood  # Возвращаем суммарное правдоподобие

def log_gradient_single_sample(feature_vector, true_label, model_parameters):
    """Градиент функции правдоподобия для одного наблюдения"""
    # Вычисляем линейную комбинацию
    linear_combination = dot_product(feature_vector, model_parameters)
    # Вычисляем предсказанную вероятность
    predicted_probability = sigmoid(linear_combination)
    # Вычисляем градиент для каждого параметра
    gradient_vector = []
    for feature_value in feature_vector:
        gradient_component = (true_label - predicted_probability) * feature_value
        gradient_vector.append(gradient_component)
    return gradient_vector  # Возвращаем вектор градиента

def log_gradient_total(feature_matrix, label_vector, model_parameters):
    """Полный градиент функции правдоподобия для всей выборки"""
    number_of_parameters = len(model_parameters)  # Количество параметров модели
    total_gradient = [0.0] * number_of_parameters  # Инициализируем градиент нулями
    # Суммируем градиенты для всех наблюдений
    for sample_index in range(len(feature_matrix)):
        # Вычисляем градиент для текущего наблюдения
        sample_gradient = log_gradient_single_sample(
            feature_matrix[sample_index], 
            label_vector[sample_index], 
            model_parameters
        )
        # Добавляем к общему градиенту
        for param_index in range(number_of_parameters):
            total_gradient[param_index] = total_gradient[param_index] + sample_gradient[param_index]
    return total_gradient  # Возвращаем полный градиент

def negate_function(original_function):
    """Возвращает функцию, которая возвращает отрицательное значение исходной"""
    def negated_function(*args, **kwargs):
        return -original_function(*args, **kwargs)  # Возвращаем отрицательное значение
    return negated_function  # Возвращаем новую функцию

In [6]:
# Обучение логистической регрессии
# Создаем оберточные функции для совместимости с gradient_descent
def loss_function_total(feature_matrix, label_vector, model_parameters):
    """Функция потерь (отрицательное правдоподобие) для всей выборки"""
    # Вычисляем отрицательное правдоподобие как функцию потерь
    negative_log_likelihood = -log_likelihood_total(feature_matrix, label_vector, model_parameters)
    return negative_log_likelihood  # Возвращаем значение функции потерь

def loss_gradient_total(feature_matrix, label_vector, model_parameters):
    """Градиент функции потерь для всей выборки"""
    # Вычисляем градиент правдоподобия
    likelihood_gradient = log_gradient_total(feature_matrix, label_vector, model_parameters)
    # Преобразуем в градиент потерь (отрицательный градиент правдоподобия)
    loss_gradient_vector = []
    for gradient_component in likelihood_gradient:
        loss_gradient_vector.append(-gradient_component)
    return loss_gradient_vector  # Возвращаем градиент функции потерь

# Улучшенная функция градиентного спуска
def improved_gradient_descent(function_to_minimize, initial_point, learning_rate=0.001, max_iterations=1000):
    """Улучшенная реализация градиентного спуска с защитой от расходимости"""
    current_point = []  # Создаем список для текущей точки
    # Копируем начальную точку
    for value in initial_point:
        current_point.append(value)
    
    best_point = []  # Создаем список для лучшей точки
    for value in current_point:
        best_point.append(value)
    best_loss = function_to_minimize(current_point)  # Вычисляем начальную функцию потерь
    
    print("Начальная функция потерь: " + "{:.4f}".format(best_loss))
    
    # Выполняем итерации градиентного спуска
    for iteration in range(max_iterations):
        # Вычисляем градиент в текущей точке
        current_gradient = gradient(function_to_minimize, current_point)
        
        # Ограничиваем норму градиента для стабильности
        gradient_norm = 0.0
        for gradient_component in current_gradient:
            gradient_norm = gradient_norm + gradient_component * gradient_component
        gradient_norm = math.sqrt(gradient_norm)
        
        # Если градиент слишком большой, нормализуем его
        if gradient_norm > 1.0:
            normalization_factor = 1.0 / gradient_norm
            for i in range(len(current_gradient)):
                current_gradient[i] = current_gradient[i] * normalization_factor
        
        # Делаем шаг в направлении, противоположном градиенту
        current_point = vector_add(current_point, scalar_multiply(-learning_rate, current_gradient))
        
        # Вычисляем текущую функцию потерь
        current_loss = function_to_minimize(current_point)
        
        # Сохраняем лучшую точку
        if current_loss < best_loss:
            best_loss = current_loss
            for i in range(len(current_point)):
                best_point[i] = current_point[i]
        
        # Выводим прогресс каждые 100 итераций
        if iteration % 100 == 0:
            print("Итерация " + str(iteration) + ", Функция потерь: " + "{:.4f}".format(current_loss))
            
        # Ранняя остановка если потери начали расти
        if current_loss > best_loss * 2.0:  # Если потери выросли в 2 раза
            print("Ранняя остановка на итерации " + str(iteration) + " - функция потерь растет")
            break
    
    print("Лучшая функция потерь: " + "{:.4f}".format(best_loss))
    return best_point  # Возвращаем лучшую найденную точку

# Инициализируем начальные значения параметров модели с меньшими значениями
number_of_features = len(train_features[0])  # Количество признаков (включая intercept)
initial_parameters = []  # Создаем список для начальных параметров
# Инициализируем параметры маленькими случайными значениями
for parameter_index in range(number_of_features):
    random_value = (random.random() - 0.5) * 0.1  # Случайные значения от -0.05 до 0.05
    initial_parameters.append(random_value)

# Форматируем начальные параметры для красивого вывода
formatted_initial_parameters = []
for parameter_value in initial_parameters:
    formatted_value = "{:.4f}".format(parameter_value)  # Форматируем до 4 знаков после запятой
    formatted_initial_parameters.append(formatted_value)
print("Начальные параметры: " + str(formatted_initial_parameters))

# Обучаем модель логистической регрессии
print("Обучение логистической регрессии с улучшенным алгоритмом...")
# Создаем функцию потерь с фиксированными обучающими данными
def loss_function_for_optimization(model_parameters):
    return loss_function_total(train_features, train_labels, model_parameters)

# Выполняем градиентный спуск для минимизации функции потерь с улучшенной функцией
optimized_parameters = improved_gradient_descent(
    loss_function_for_optimization,  # Функция для минимизации
    initial_parameters,              # Начальные значения параметров
    learning_rate=0.001,            # Малая скорость обучения для стабильности
    max_iterations=2000             # Максимальное количество итераций
)

# Форматируем обученные параметры для красивого вывода
formatted_optimized_parameters = []
for parameter_value in optimized_parameters:
    formatted_value = "{:.4f}".format(parameter_value)  # Форматируем до 4 знаков после запятой
    formatted_optimized_parameters.append(formatted_value)
print("Обученные параметры: " + str(formatted_optimized_parameters))

Начальные параметры: ['-0.0144', '-0.0127', '-0.0279', '0.0281', '0.0406', '0.0089', '-0.0242', '-0.0079', '-0.0148', '0.0494', '-0.0421', '0.0465']
Обучение логистической регрессии с улучшенным алгоритмом...
Начальная функция потерь: 435.4645
Итерация 0, Функция потерь: 435.0939
Итерация 100, Функция потерь: 400.3084
Итерация 200, Функция потерь: 369.9326
Итерация 300, Функция потерь: 343.7524
Итерация 400, Функция потерь: 321.4706
Итерация 500, Функция потерь: 302.7375
Итерация 600, Функция потерь: 287.1790
Итерация 700, Функция потерь: 274.4205
Итерация 800, Функция потерь: 264.1024
Итерация 900, Функция потерь: 255.8905
Итерация 1000, Функция потерь: 249.4812
Итерация 1100, Функция потерь: 244.6030
Итерация 1200, Функция потерь: 241.0165
Итерация 1300, Функция потерь: 238.5145
Итерация 1400, Функция потерь: 236.9201
Итерация 1500, Функция потерь: 236.0856
Итерация 1600, Функция потерь: 235.8746
Итерация 1700, Функция потерь: 235.8739
Итерация 1800, Функция потерь: 235.8739
Итерация

#### ЗАДАНИЕ 4-5: ПРЕДСКАЗАНИЯ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ И ВЫДЕЛЕНИЕ НЕСТАБИЛЬНЫХ СЛУЧАЕВ

In [7]:
# Предсказания с помощью логистической регрессии
def predict_logistic_regression(feature_vector, model_parameters, classification_threshold=0.5):
    """Предсказание метки с помощью логистической регрессии"""
    # Вычисляем вероятность принадлежности к классу 1
    predicted_probability = sigmoid(dot_product(feature_vector, model_parameters))
    # Применяем порог для классификации
    if predicted_probability >= classification_threshold:
        return 1  # Класс 1 (есть заболевание сердца)
    else:
        return 0  # Класс 0 (нет заболевания сердца)

def predict_logistic_regression_with_uncertainty(feature_vector, model_parameters, low_threshold=0.3, high_threshold=0.7):
    """Предсказание с выделением нестабильных случаев с более широкой серой зоной"""
    # Вычисляем вероятность принадлежности к классу 1
    predicted_probability = sigmoid(dot_product(feature_vector, model_parameters))
    # Определяем класс на основе вероятности и порогов
    if predicted_probability >= low_threshold and predicted_probability <= high_threshold:
        return 2  # Нестабильный случай (требует дополнительной проверки)
    elif predicted_probability > high_threshold:
        return 1  # Класс 1 (высокая уверенность в наличии заболевания)
    else:
        return 0  # Класс 0 (высокая уверенность в отсутствии заболевания)

# Делаем предсказания на тестовой выборке с помощью логистической регрессии
logistic_regression_predictions_binary = []  # Список для бинарных предсказаний
for test_feature_vector in test_features:
    # Предсказываем метку для каждого тестового примера
    binary_prediction = predict_logistic_regression(test_feature_vector, optimized_parameters)
    logistic_regression_predictions_binary.append(binary_prediction)

logistic_regression_predictions_ternary = []  # Список для тернарных предсказаний
for test_feature_vector in test_features:
    # Предсказываем метку с учетом нестабильных случаев
    ternary_prediction = predict_logistic_regression_with_uncertainty(test_feature_vector, optimized_parameters)
    logistic_regression_predictions_ternary.append(ternary_prediction)

# Анализируем нестабильные предсказания
unstable_predictions_count = 0  # Счетчик нестабильных предсказаний
for prediction_value in logistic_regression_predictions_ternary:
    if prediction_value == 2:  # Если предсказание нестабильное
        unstable_predictions_count = unstable_predictions_count + 1

total_predictions_count = len(logistic_regression_predictions_ternary)  # Общее количество предсказаний
# Вычисляем процент нестабильных предсказаний
unstable_percentage = (unstable_predictions_count / total_predictions_count) * 100

# Выводим статистику по нестабильным предсказаниям
print("Нестабильных предсказаний: " + str(unstable_predictions_count) + "/" + str(total_predictions_count) + 
      " (" + "{:.2f}".format(unstable_percentage) + "%)")

# Анализируем распределение вероятностей для отладки
print("\nАнализ распределения вероятностей:")
probability_values = []
for test_feature_vector in test_features:
    probability = sigmoid(dot_product(test_feature_vector, optimized_parameters))
    probability_values.append(probability)

min_prob = min(probability_values)
max_prob = max(probability_values)
mean_prob = sum(probability_values) / len(probability_values)
print("Минимальная вероятность: " + "{:.4f}".format(min_prob))
print("Максимальная вероятность: " + "{:.4f}".format(max_prob))
print("Средняя вероятность: " + "{:.4f}".format(mean_prob))

Нестабильных предсказаний: 50/275 (18.18%)

Анализ распределения вероятностей:
Минимальная вероятность: 0.0107
Максимальная вероятность: 0.9968
Средняя вероятность: 0.5472


#### ЗАДАНИЕ 6: РЕАЛИЗАЦИЯ МЕТОДА БЛИЖАЙШИХ СОСЕДЕЙ

In [8]:
# Реализация метода ближайших соседей
def euclidean_distance(vector1, vector2):
    """Вычисляет евклидово расстояние между двумя векторами одинаковой длины"""
    sum_squared_differences = 0.0  # Инициализируем сумму квадратов разностей
    # Проходим по всем элементам векторов
    for element_index in range(len(vector1)):
        # Вычисляем разность между соответствующими элементами
        difference = vector1[element_index] - vector2[element_index]
        # Добавляем квадрат разности к сумме
        sum_squared_differences = sum_squared_differences + difference * difference
    # Извлекаем квадратный корень из суммы квадратов разностей
    distance = math.sqrt(sum_squared_differences)
    return distance  # Возвращаем евклидово расстояние

def find_k_nearest_neighbors(train_features_matrix, train_labels_vector, test_instance, number_of_neighbors):
    """Находит k ближайших соседей для тестового примера"""
    distances_list = []  # Создаем список для хранения расстояний
    # Вычисляем расстояния до всех обучающих примеров
    for train_index in range(len(train_features_matrix)):
        train_feature_vector = train_features_matrix[train_index]  # Признаки текущего обучающего примера
        train_label_value = train_labels_vector[train_index]        # Метка текущего обучающего примера
        # Вычисляем расстояние от тестового примера до текущего обучающего
        current_distance = euclidean_distance(test_instance, train_feature_vector)
        # Сохраняем пример, его метку и расстояние
        distances_list.append((train_feature_vector, train_label_value, current_distance))
    
    # Сортируем список по расстоянию (от ближайшего к дальнему)
    def get_distance(element):
        return element[2]  # Возвращаем расстояние для сортировки
    distances_list.sort(key=get_distance)
    
    # Выбираем k ближайших соседей
    nearest_neighbors = []
    for neighbor_index in range(number_of_neighbors):
        nearest_neighbors.append(distances_list[neighbor_index])
    return nearest_neighbors  # Возвращаем список ближайших соседей

def predict_k_nearest_neighbors(train_features_matrix, train_labels_vector, test_instance, number_of_neighbors=5):
    """Предсказание метки с помощью метода k ближайших соседей"""
    # Находим k ближайших соседей
    neighbors_list = find_k_nearest_neighbors(train_features_matrix, train_labels_vector, test_instance, number_of_neighbors)
    
    votes_dictionary = {}  # Создаем словарь для подсчета голосов
    # Подсчитываем голоса соседей
    for neighbor_tuple in neighbors_list:
        neighbor_label = neighbor_tuple[1]  # Получаем метку соседа
        # Увеличиваем счетчик голосов для этой метки
        if neighbor_label in votes_dictionary:
            votes_dictionary[neighbor_label] = votes_dictionary[neighbor_label] + 1
        else:
            votes_dictionary[neighbor_label] = 1
    
    # Находим метку с наибольшим числом голосов
    best_label = None
    max_votes = -1
    for label_value, vote_count in votes_dictionary.items():
        if vote_count > max_votes:
            max_votes = vote_count
            best_label = label_value
    
    return best_label  # Возвращаем метку с наибольшим числом голосов

# Подготавливаем данные для KNN (убираем intercept, так как он не нужен для расстояний)
train_features_for_knn = []  # Признаки обучающей выборки для KNN
for train_feature_vector in train_features:
    feature_vector_without_intercept = []
    # Пропускаем первый элемент (intercept)
    for feature_index in range(1, len(train_feature_vector)):
        feature_vector_without_intercept.append(train_feature_vector[feature_index])
    train_features_for_knn.append(feature_vector_without_intercept)

test_features_for_knn = []  # Признаки тестовой выборки для KNN
for test_feature_vector in test_features:
    feature_vector_without_intercept = []
    # Пропускаем первый элемент (intercept)
    for feature_index in range(1, len(test_feature_vector)):
        feature_vector_without_intercept.append(test_feature_vector[feature_index])
    test_features_for_knn.append(feature_vector_without_intercept)

# Делаем предсказания методом ближайших соседей
print("Применение метода ближайших соседей...")
knn_predictions = []  # Список для предсказаний KNN
for test_feature_vector in test_features_for_knn:
    # Предсказываем метку для каждого тестового примера
    knn_prediction = predict_k_nearest_neighbors(train_features_for_knn, train_labels, test_feature_vector, number_of_neighbors=5)
    knn_predictions.append(knn_prediction)

print("Предсказания KNN завершены")

Применение метода ближайших соседей...
Предсказания KNN завершены


#### ЗАДАНИЕ 7: ВЫЧИСЛЕНИЕ МЕТРИК КАЧЕСТВА КЛАССИФИКАЦИИ

In [9]:
# Вычисление метрик качества
def calculate_binary_classification_metrics(true_labels_vector, predicted_labels_vector):
    """Вычисление метрик для бинарной классификации"""
    # Инициализируем счетчики матрицы ошибок
    true_positives = 0  # Истинные положительные (правильно предсказанные больные)
    false_positives = 0 # Ложные положительные (здоровые, предсказанные как больные)
    true_negatives = 0  # Истинные отрицательные (правильно предсказанные здоровые)
    false_negatives = 0 # Ложные отрицательные (больные, предсказанные как здоровые)
    
    # Подсчитываем элементы матрицы ошибок
    for sample_index in range(len(true_labels_vector)):
        true_label = true_labels_vector[sample_index]
        predicted_label = predicted_labels_vector[sample_index]
        if true_label == 1 and predicted_label == 1:
            true_positives = true_positives + 1
        elif true_label == 1 and predicted_label == 0:
            false_negatives = false_negatives + 1
        elif true_label == 0 and predicted_label == 1:
            false_positives = false_positives + 1
        elif true_label == 0 and predicted_label == 0:
            true_negatives = true_negatives + 1
    
    # Вычисляем метрики качества (с проверкой деления на ноль)
    if (true_positives + false_positives) > 0:
        precision_metric = true_positives / (true_positives + false_positives)
    else:
        precision_metric = 0.0
    
    if (true_positives + false_negatives) > 0:
        recall_metric = true_positives / (true_positives + false_negatives)
    else:
        recall_metric = 0.0
    
    if (precision_metric + recall_metric) > 0:
        f1_score_metric = 2 * precision_metric * recall_metric / (precision_metric + recall_metric)
    else:
        f1_score_metric = 0.0
    
    if len(true_labels_vector) > 0:
        accuracy_metric = (true_positives + true_negatives) / len(true_labels_vector)
    else:
        accuracy_metric = 0.0
    
    # Возвращаем словарь с вычисленными метриками
    metrics_dictionary = {
        'precision': precision_metric,
        'recall': recall_metric,
        'f1': f1_score_metric,
        'accuracy': accuracy_metric,
        'true_positives': true_positives,
        'false_positives': false_positives,
        'true_negatives': true_negatives,
        'false_negatives': false_negatives
    }
    return metrics_dictionary

def calculate_ternary_classification_accuracy(true_labels_vector, ternary_predictions_vector):
    """Точность для тернарной классификации (игнорируя нестабильные случаи)"""
    correct_predictions_count = 0  # Счетчик правильных предсказаний
    total_stable_predictions_count = 0  # Счетчик стабильных предсказаний (исключая нестабильные)
    
    # Подсчитываем правильные предсказания только для стабильных случаев
    for sample_index in range(len(true_labels_vector)):
        ternary_prediction = ternary_predictions_vector[sample_index]
        # Игнорируем нестабильные предсказания (класс 2)
        if ternary_prediction != 2:
            total_stable_predictions_count = total_stable_predictions_count + 1
            true_label = true_labels_vector[sample_index]
            if true_label == ternary_prediction:
                correct_predictions_count = correct_predictions_count + 1
                
    # Вычисляем точность для стабильных предсказаний
    if total_stable_predictions_count > 0:
        ternary_accuracy = correct_predictions_count / total_stable_predictions_count
        # Выводим дополнительную информацию
        print("Стабильные предсказания: " + str(total_stable_predictions_count) + "/" + str(len(true_labels_vector)))
        print("Правильные стабильные предсказания: " + str(correct_predictions_count) + "/" + str(total_stable_predictions_count))
    else:
        ternary_accuracy = 0.0
        print("Нет стабильных предсказаний для вычисления точности")
        
    return ternary_accuracy  # Возвращаем точность

# Вычисляем метрики для логистической регрессии (бинарная классификация)
logistic_regression_binary_metrics = calculate_binary_classification_metrics(test_labels, logistic_regression_predictions_binary)

# Вычисляем точность для тернарной классификации логистической регрессии
print("\n--- ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ (ТЕРНАРНАЯ) ---")
logistic_regression_ternary_accuracy = calculate_ternary_classification_accuracy(test_labels, logistic_regression_predictions_ternary)

# Вычисляем метрики для метода ближайших соседей
knn_metrics = calculate_binary_classification_metrics(test_labels, knn_predictions)

# Выводим результаты вычисления метрик
print("\n--- МЕТРИКИ КАЧЕСТВА ---")
print("Логистическая регрессия (бинарная):")
print("  Precision: " + "{:.4f}".format(logistic_regression_binary_metrics['precision']))
print("  Recall:    " + "{:.4f}".format(logistic_regression_binary_metrics['recall']))
print("  F1-score:  " + "{:.4f}".format(logistic_regression_binary_metrics['f1']))
print("  Accuracy:  " + "{:.4f}".format(logistic_regression_binary_metrics['accuracy']))
print("  TP: " + str(logistic_regression_binary_metrics['true_positives']) + 
      ", FP: " + str(logistic_regression_binary_metrics['false_positives']) +
      ", TN: " + str(logistic_regression_binary_metrics['true_negatives']) +
      ", FN: " + str(logistic_regression_binary_metrics['false_negatives']))

print("\nЛогистическая регрессия (тернарная):")
print("  Accuracy (без нестабильных): " + "{:.4f}".format(logistic_regression_ternary_accuracy))

print("\nМетод ближайших соседей (k=5):")
print("  Precision: " + "{:.4f}".format(knn_metrics['precision']))
print("  Recall:    " + "{:.4f}".format(knn_metrics['recall']))
print("  F1-score:  " + "{:.4f}".format(knn_metrics['f1']))
print("  Accuracy:  " + "{:.4f}".format(knn_metrics['accuracy']))
print("  TP: " + str(knn_metrics['true_positives']) + 
      ", FP: " + str(knn_metrics['false_positives']) +
      ", TN: " + str(knn_metrics['true_negatives']) +
      ", FN: " + str(knn_metrics['false_negatives']))


--- ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ (ТЕРНАРНАЯ) ---
Стабильные предсказания: 225/275
Правильные стабильные предсказания: 204/225

--- МЕТРИКИ КАЧЕСТВА ---
Логистическая регрессия (бинарная):
  Precision: 0.8800
  Recall:    0.8684
  F1-score:  0.8742
  Accuracy:  0.8618
  TP: 132, FP: 18, TN: 105, FN: 20

Логистическая регрессия (тернарная):
  Accuracy (без нестабильных): 0.9067

Метод ближайших соседей (k=5):
  Precision: 0.8947
  Recall:    0.8947
  F1-score:  0.8947
  Accuracy:  0.8836
  TP: 136, FP: 16, TN: 107, FN: 16


#### ЗАДАНИЕ 8: ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ И ВЫВОДЫ

In [10]:
# Интерпретация результатов
print("\n--- ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ ---")
print("1. Сравнение методов классификации:")
print("   - Логистическая регрессия показывает Accuracy = " + 
      "{:.4f}".format(logistic_regression_binary_metrics['accuracy']))
print("   - Метод ближайших соседей показывает Accuracy = " + 
      "{:.4f}".format(knn_metrics['accuracy']))

print("\n2. Анализ нестабильных предсказаний:")
print("   - " + "{:.2f}".format(unstable_percentage) + 
      "% примеров классифицируются как нестабильные")
print("   - Эти случаи находятся в 'серой зоне' и требуют дополнительного внимания врача")
print("   - Нестабильные случаи могут указывать на пограничные состояния пациентов")

print("\n3. Качество классификации по метрикам:")
print("   - Precision: доля правильно предсказанных больных среди всех предсказанных больных")
print("   - Recall: доля правильно найденных больных среди всех реально больных") 
print("   - F1-score: гармоническое среднее между Precision и Recall")
print("   - Accuracy: общая доля правильных предсказаний")

print("\n4. Выводы о методах классификации:")
print("   - Оба метода демонстрируют сопоставимое качество классификации")
print("   - Логистическая регрессия более интерпретируема - можно анализировать веса признаков")
print("   - Метод ближайших соседей не требует обучения, но медленнее на этапе предсказания")
print("   - Введение третьего класса (нестабильные) повышает надежность системы")
print("   - Для критических медицинских решений важна не только точность, но и интерпретируемость")


--- ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ ---
1. Сравнение методов классификации:
   - Логистическая регрессия показывает Accuracy = 0.8618
   - Метод ближайших соседей показывает Accuracy = 0.8836

2. Анализ нестабильных предсказаний:
   - 18.18% примеров классифицируются как нестабильные
   - Эти случаи находятся в 'серой зоне' и требуют дополнительного внимания врача
   - Нестабильные случаи могут указывать на пограничные состояния пациентов

3. Качество классификации по метрикам:
   - Precision: доля правильно предсказанных больных среди всех предсказанных больных
   - Recall: доля правильно найденных больных среди всех реально больных
   - F1-score: гармоническое среднее между Precision и Recall
   - Accuracy: общая доля правильных предсказаний

4. Выводы о методах классификации:
   - Оба метода демонстрируют сопоставимое качество классификации
   - Логистическая регрессия более интерпретируема - можно анализировать веса признаков
   - Метод ближайших соседей не требует обучения, но медленне